# Importing the libraries

##Installs

In [1]:
pip install pickle5

     |████████████████████████████████| 133kB 5.1MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp36-cp36m-linux_x86_64.whl size=218623 sha256=dcbd91768341bd24964a3970f2ab5d3f783ffad428f455d25f74347b45539994
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


## Core libraries

In [2]:
import numpy
import tensorflow as tf
import random
import json
import pickle5 as pickle

## Importing nltk

In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Importing from Keras 

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

# Code

In [8]:
with open("intents.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]
 
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

#dumping dataset into data.pickle
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


## Training the Deep Learning Model

In [9]:
tf.compat.v1.reset_default_graph()

# define the keras model
model = Sequential()
model.add(Dense(8, activation='relu', input_shape=(len(training[0]),)))
model.add(Dense(8, activation="relu"))
model.add(Dense(len(output[0]), activation="softmax"))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
history = model.fit(training,output, epochs=1000, batch_size=20)

Epoch 1/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.1400 - accuracy: 0.1538
Epoch 2/1000
2/2 [==============================] - 0s 2ms/step - loss: 0.1397 - accuracy: 0.1538
Epoch 3/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.1394 - accuracy: 0.1538
Epoch 4/1000
2/2 [==============================] - 0s 1ms/step - loss: 0.1391 - accuracy: 0.1538
Epoch 5/1000
2/2 [==============================] - 0s 2ms/step - loss: 0.1388 - accuracy: 0.1538
Epoch 6/1000
2/2 [==============================] - 0s 1ms/step - loss: 0.1386 - accuracy: 0.1923
Epoch 7/1000
2/2 [==============================] - 0s 1ms/step - loss: 0.1383 - accuracy: 0.1923
Epoch 8/1000
2/2 [==============================] - 0s 1ms/step - loss: 0.1380 - accuracy: 0.1923
Epoch 9/1000
2/2 [==============================] - 0s 2ms/step - loss: 0.1378 - accuracy: 0.1923
Epoch 10/1000
2/2 [==============================] - 0s 1ms/step - loss: 0.1376 - accuracy: 0.1923
Epoch 11/1000
2/2 [

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 376       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 54        
Total params: 502
Trainable params: 502
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.save('chatbot_model.h5')

##Calling the ChatBot

In [11]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [12]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict(numpy.array([bag_of_words(inp,words)]))
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        arr = []
        for x in results:
          arr.extend(x)

        for tg in data["intents"]:
              if tg['tag'] == tag:
                  responses = tg['responses']
                  print(random.choice(responses))


##Run this cell to chat :)

In [13]:
chat()

Start talking with the bot (type quit to stop)!
You: Hi
Good to see you again!
You: quit
